In [1]:
import glob
import mne
import pickle


if not glob.glob('./all_epochs.pickle'):
    all_epochs = None
    p_folders = glob.glob('./dataset_hackathon/P*/')
    for p_folder in p_folders:
        s_folders = glob.glob(f'{p_folder}S*')
        for s_folder in s_folders:
            files = glob.glob(f'{s_folder}/eeg/*MAT*.set')
            for f in files:
                epochs = mne.io.read_epochs_eeglab(f, verbose=False)  # FIR 1-40 Hz
                if all_epochs is None:
                    all_epochs = epochs.copy()
                else:
                    all_epochs = mne.concatenate_epochs([all_epochs, epochs.copy()], verbose=False)

    with open('all_epochs.pickle', 'wb') as w:
        pickle.dump(all_epochs, w)

else:
    with open('all_epochs.pickle', 'rb') as r:
        all_epochs = pickle.load(r)

all_epochs_numpy = all_epochs.get_data(copy=True)
print(all_epochs_numpy.shape)

(13410, 61, 500)


In [2]:
X_data = all_epochs_numpy[:,:,:450]
y_data = all_epochs_numpy[:,:,50:]

In [3]:
import torch

In [4]:
y_data_t = y_data.transpose(2, 0, 1)
x_data_t = X_data.transpose(2, 0, 1)

In [5]:
print(x_data_t.shape)
print(y_data_t.shape)

(450, 13410, 61)
(450, 13410, 61)


In [6]:
x_numeric = torch.tensor(x_data_t, dtype=torch.float32)
y_numeric = torch.tensor(y_data_t, dtype=torch.float32)

In [7]:
x_numeric.shape

torch.Size([450, 13410, 61])

In [ ]:
model = torch.nn.Transformer(d_model=61, nhead=1, batch_first=False)

In [9]:
out = model(x_numeric[:, :4, :], y_numeric[:, :4, :])
out.shape

torch.Size([450, 4, 61])